# **Modeling the Impact of Wheather on Water Consumption in Barcelona** 

## **Imports**

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from datetime import datetime
from catboost import CatBoostRegressor
from sklearn.model_selection import TimeSeriesSplit
import pickle

In [ ]:
# Load dataset
data_path = '../data'

df = pd.read_csv(os.path.join(data_path, 'consumption_weather.csv'), parse_dates=['FECHA'])
print(df.shape)
df.head()

## **Feature Engineering**

In [ ]:
lista_festivos = [
    # 2021
    "2021-01-01",  
    "2021-01-06", 
    "2021-04-02", 
    "2021-04-05",  
    "2021-05-01",  
    "2021-05-24",  
    "2021-06-24",  
    "2021-08-15",  
    "2021-09-11",  
    "2021-09-24",  
    "2021-10-12",  
    "2021-11-01",  
    "2021-12-06",  
    "2021-12-08",  
    "2021-12-25",  
    "2021-12-26",  

    # 2022
    "2022-01-01",  
    "2022-01-06",  
    "2022-04-15",  
    "2022-04-18",  
    "2022-06-06",  
    "2022-06-24",  
    "2022-08-15",  
    "2022-09-24",  
    "2022-10-12",  
    "2022-11-01",  
    "2022-12-06",  
    "2022-12-08",  
    "2022-12-25",  
    "2022-12-26",  

    # 2023
    "2023-01-06",  
    "2023-04-07",  
    "2023-04-10",
    "2023-06-05",  
    "2023-06-24",  
    "2023-08-15",  
    "2023-09-11",  
    "2023-09-25",  
    "2023-10-12",  
    "2023-11-01",  
    "2023-12-06",  
    "2023-12-08",  
    "2023-12-25",  
    "2023-12-26",  

    # 2024
    "2024-01-01",
    "2024-01-06",  
    "2024-03-29",  
    "2024-04-01",  
    "2024-05-01",  
    "2024-05-20",  
    "2024-06-24",  
    "2024-08-15",  
    "2024-09-11",  
    "2024-09-24",  
    "2024-10-12",  
    "2024-11-01",  
    "2024-12-06",  
    "2024-12-25",  
    "2024-12-26",  
]
lista_festivos = pd.to_datetime(lista_festivos)

In [ ]:
weather_col_names = ['WindDir_Mean_10m', 'WindDir_Max_10m', 
    'Humidity_Mean', 'Humidity_Min', 'Humidity_Max', 
    'Pressure_Mean', 'Pressure_Min', 'Precipitation', 
    'Pressure_Max', 'Solar_Radiation_24h', 'Temp_Mean', 
    'Temp_Min', 'Temp_Max', 'Temp_App', 'WindSpeed_Mean_10m', 'WindSpeed_Max_10m', 'dry', 'dry_streak']

col_names = [
    'month',
    'dayofweek',
    'day',
    'weekofyear',
    'year',
    'is_weekend',
    'is_holiday',
    'is_summer',
    'CONSUMO_LAG1',
    'CONSUMO_LAG3',
    'CONSUMO_LAG7',
    'CONSUMO_LAG14',
    'CONSUMO_LAG30',
    'CONSUMO_ROLL7',
    'CONSUMO_STDROLL7',
    'CONSUMO_ROLL14',
    'CONSUMO_ROLL30',
    'CONSUMO_STDROLL30',
    'DIFF1',
    'DIFF7']

In [ ]:
# Feature Engineering

df["Temp_App"] = df["Temp_Mean"] + 0.33*df["Humidity_Mean"]/100 - 0.70*df["WindSpeed_Mean_10m"] - 4

df['month'] = df['FECHA'].dt.month # month
df['dayofweek'] = df['FECHA'].dt.dayofweek # Monday=0, Sunday=6
df['day'] = df['FECHA'].dt.day # day of month
df["weekofyear"] = df["FECHA"].dt.isocalendar().week.astype(int) # week of year
df['year'] = df['FECHA'].dt.year # year
df['is_weekend'] = df['dayofweek'] >= 5 # weekend indicator
df["is_holiday"] = df["FECHA"].isin(lista_festivos).astype(int) # holiday indicator
df["is_summer"] = df["FECHA"].dt.month.isin([6,7,8]).astype(int) # summer months

for lag in [1,3,7,14,30]:
    df[f"CONSUMO_LAG{lag}"] = df["CONSUM_DIARI"].shift(lag) # lagged consumption

for roll in [7,14,30]:
    df[f"CONSUMO_ROLL{roll}"] = df["CONSUM_DIARI"].rolling(window=roll).mean()
    if roll != 14: 
        df[f"CONSUMO_STDROLL{roll}"] = df["CONSUM_DIARI"].rolling(window=roll).std()

df["DIFF1"] = df["CONSUM_DIARI"].diff()
df["DIFF7"] = df["CONSUM_DIARI"] - df["CONSUMO_ROLL7"] # difference from 7-day rolling mean

df["dry"] = (df["Precipitation"] == 0).astype(int) # 1 if no precipitation, 0 otherwise
df["dry_streak"] = df["dry"].groupby((df["dry"] != df["dry"].shift()).cumsum()).cumsum() # count consecutive dry days

    
df = df.dropna()

df.head()

## **Model**

In [ ]:
model = CatBoostRegressor(
    depth=8,
    learning_rate=0.05,
    loss_function="MAE",
    iterations=1500,
    random_seed=42,
    verbose=200
)

In [ ]:
# Feature definition

target = 'CONSUM_DIARI'

# Train-test split

X = df[col_names + weather_col_names]
y = df[target]

tscv = TimeSeriesSplit(n_splits=5)

for train_index, val_index in tscv.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

os.makedirs("../streamlit_app/models", exist_ok=True)
model.save_model("../streamlit_app/models/catboost_weather.cbm")
print("Model with weather features saved.")

In [ ]:
with open(os.path.join(data_path, 'consumption_weather_test.pkl'), 'wb') as fp:
    pickle.dump((X_val, y_val), fp)

In [ ]:
def smape(y_true, y_pred):
    return 100 * np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

In [ ]:
# Evaluate model
y_pred = model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
smape_value = smape(y_val, y_pred)

print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}, SMAPE: {smape_value:.2f}")

# Plot results
plt.figure(figsize=(10,5))
plt.plot(df["FECHA"].iloc[-len(y_val):], y_val, label="Actual")
plt.plot(df["FECHA"].iloc[-len(y_val):], y_pred, label="Predicted")
plt.legend()
plt.title("Predicted vs Actual Daily Consumption")
plt.show()

In [ ]:
df["Predicted"] = model.predict(X)

plt.figure(figsize=(12,6))
plt.plot(df["FECHA"], df["CONSUM_DIARI"], label="Actual", linewidth=2)
plt.plot(df["FECHA"], df["Predicted"], label="Predicted", alpha=0.7)
plt.title("Actual vs Predicted Daily Consumption (Full Period)")
plt.xlabel("Date")
plt.ylabel("Consumption")
plt.legend()
plt.show()


In [ ]:
confuding_matrix = pd.DataFrame({
    'Actual': y_val,
    'Predicted': y_pred
})

plt.figure(figsize=(8,6))
sns.scatterplot(data=confuding_matrix, x='Actual', y='Predicted', alpha=0.6)
plt.plot([confuding_matrix['Actual'].min(), confuding_matrix['Actual'].max()],
         [confuding_matrix['Actual'].min(), confuding_matrix['Actual'].max()],
         color='red', linestyle='--')
plt.title("Actual vs Predicted Scatter Plot")
plt.xlabel("Actual Consumption")
plt.ylabel("Predicted Consumption")
plt.show()

In [ ]:
corr = df.corr(numeric_only=True)

print(corr)

target_corr = corr['CONSUM_DIARI'].sort_values(ascending=False)
print(target_corr)

plt.figure(figsize=(6, 5))
plt.imshow(corr, cmap='coolwarm', interpolation='nearest')
plt.colorbar()
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Correlation Matrix")
plt.tight_layout()
plt.show()


In [ ]:
# try without the weather features. 

X = df[col_names]
y = df[target]

tscv = TimeSeriesSplit(n_splits=5)

for train_index, val_index in tscv.split(X):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)

model.save_model("../streamlit_app/models/catboost_no_weather.cbm")
print("Model without weather features saved.")

In [ ]:
y_pred = model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
smape_value = smape(y_val, y_pred)

print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R²: {r2:.3f}, SMAPE: {smape_value:.2f}")

plt.figure(figsize=(10,5))
plt.plot(df["FECHA"].iloc[-len(y_val):], y_val, label="Actual")
plt.plot(df["FECHA"].iloc[-len(y_val):], y_pred, label="Predicted")
plt.legend()
plt.title("Predicted vs Actual Daily Consumption")
plt.show()